<a href="https://colab.research.google.com/github/Dax1805/cs6200-documents/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
## If you already have gensim, you can comment this out.
!pip install gensim
!pip uninstall -y numpy
!pip cache purge
!pip install --force-reinstall numpy==1.26.0
!pip install --upgrade pip setuptools

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0
Files removed: 18 (21.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 127.8 MB/s eta 0:00:00


In [2]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [3]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2025-03-25 10:20:41--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  --.-KB/s    in 0.1s    

2025-03-25 10:20:41 (35.1 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [4]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks we discussed in class.

In [5]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.7983104586601257),
 ('prince', 0.7396013736724854),
 ('york', 0.7378042340278625),
 ('princess', 0.7186989784240723),
 ('duke', 0.7092494368553162),
 ('clarence', 0.6882430911064148),
 ('france', 0.677958071231842),
 ('suffolk', 0.6764191389083862),
 ('warwick', 0.6740710139274597),
 ('gloucester', 0.665603518486023)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [6]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9631298184394836),
 ('iago', 0.937802791595459),
 ('emilia', 0.9329056143760681),
 ('cassio', 0.9213374257087708),
 ('ham', 0.9139888882637024),
 ('imogen', 0.9128400087356567),
 ('cleopatra', 0.9081115126609802),
 ('lucio', 0.9041749238967896),
 ('pisanio', 0.9036695957183838),
 ('troilus', 0.8940178155899048)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [7]:
model.wv.similarity('othello', 'desdemona')

0.9631298

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [8]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  A = np.dot(v1, v2)

  B = np.linalg.norm(v1)
  C = np.linalg.norm(v2)

  return A / (B * C)

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.96312994

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [21]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2025-03-25 11:11:03--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M   108MB/s    in 0.6s    

2025-03-25 11:11:04 (108 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2025-03-25 11:11:04--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.1’

30k.fr.vec.1        100%[===================>]  64.66M  90.6MB/s    in 0.7s    

2025-03-25 11:11:05 (90.6 MB/s) - ‘30k.fr.vec.1’ saved [67802327/67802327]

--2025-03-25 11:11:05--  http://www.c

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [23]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1}

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this part of the assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [24]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
    best_match = None
    best_score = 0

    for word, candidate_vector in vecDict.items():
        current_score = cosim(vec, candidate_vector)
        if current_score > best_score:
            best_score = current_score
            best_match = word

    return (best_match, best_score)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [25]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2025-03-25 11:11:40--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.1’

links.tab.1         100%[===================>]   1.34M  --.-KB/s    in 0.04s   

2025-03-25 11:11:40 (31.8 MB/s) - ‘links.tab.1’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [26]:
links[302]

['academy', 'fr', 'académie']

We construct a test set for English-French from the first 1000 links between those languages.

In [27]:
frtest = [x for x in links if x[1] == "fr"][0:1000]
frtest[0:10]

[['aalborg', 'fr', 'aalborg'],
 ['aarhus', 'fr', 'aarhus'],
 ['aba', 'fr', 'aba'],
 ['abad', 'fr', 'abad'],
 ['abandon', 'fr', 'abandon'],
 ['abbas', 'fr', 'abbas'],
 ['abbasid', 'fr', 'abbassides'],
 ['abbess', 'fr', 'abbesse'],
 ['abbey', 'fr', 'abbaye'],
 ['abbot', 'fr', 'abbé']]

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article in this test set `frtest` is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary.

Since many articles, e.g., about named entities, have the same title in English and French, use the identity function as a baseline and compute its accuracy. In other words, how often would you find the right French articles by simply echoing the English title as if it were French? In the ten example links above, `aalborg` and `aarhus` (two cities in Denmark) are the same in English and French. Remember to iterate only over the 1000 linked Wikipedia articles in the test set, not the entire embedding dictionary.

In [28]:
## TODO: Compute English-French Wikipedia retrieval accuracy.

correct_retrievals = 0
correct_identity = 0
eligible_count = 0

# Iterate over the first 1000 French-linked Wikipedia articles.
for link_entry in frtest:
    eng_title = link_entry[0]
    fr_title = link_entry[2]

    # Skip if the article isn't in the embedding dictionary.
    if eng_title not in envec:
        continue
    if fr_title not in frvec:
        continue

    eligible_count += 1
    predicted_fr, sim_score = mostSimilar(envec[eng_title], frvec)

    # Check if the predicted French title matches the actual French title.
    if predicted_fr == fr_title:
        correct_retrievals += 1

    # Check if the English title is the same as the French title.
    if eng_title == fr_title:
        correct_identity += 1

# Compute the proportions.
accuracy = correct_retrievals / eligible_count if eligible_count else 0
baselineAccuracy = correct_identity / eligible_count if eligible_count else 0

print("Embedding-based retrieval accuracy:", accuracy)
print("Baseline (identity) accuracy:", baselineAccuracy)

Embedding-based retrieval accuracy: 0.8443465491923642
Baseline (identity) accuracy: 0.6064610866372981


**TODO**: Compute the accuracy, i.e. precision@1, of the embeddings and of the identity function for the first 1000 links between English and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [29]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
# Filter the first 1000 links for Spanish
estest = [entry for entry in links if entry[1] == "es"][:1000]

correct_retrievals = 0
correct_identity = 0
eligible_count = 0

for entry in estest:
    eng_title = entry[0]
    es_title = entry[2]

    # Skip entries where either title isn't in the respective embedding dictionary.
    if eng_title not in envec or es_title not in esvec:
        continue

    eligible_count += 1
    # Predict the Spanish title using the embedding-based nearest neighbor search.
    predicted, _ = mostSimilar(envec[eng_title], esvec)

    if predicted == es_title:
        correct_retrievals += 1
    if eng_title == es_title:
        correct_identity += 1

# Compute precision@1
accuracy = correct_retrievals / eligible_count if eligible_count else 0
baselineAccuracy = correct_identity / eligible_count if eligible_count else 0

print("Embedding-based retrieval accuracy:", accuracy)
print("Baseline (identity) accuracy:", baselineAccuracy)

Embedding-based retrieval accuracy: 0.8284389489953632
Baseline (identity) accuracy: 0.45904173106646057


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [33]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.
!pip install --upgrade faiss-cpu
import faiss
import numpy as np

# --- French FAISS index ---
french_words = list(frvec.keys())
french_matrix = np.stack([frvec[word] for word in french_words]).astype('float32')
faiss.normalize_L2(french_matrix)
index_fr = faiss.IndexFlatIP(french_matrix.shape[1])
index_fr.add(french_matrix)

# Evaluate recall@10 and MRR@10 using FAISS for French links in frtest.
correct_retrievals = 0
eligible_count = 0
mrr_sum = 0.0

for entry in frtest:
    eng_title = entry[0]
    fr_title = entry[2]

    if eng_title not in envec or fr_title not in frvec:
        continue
    eligible_count += 1

    query_vec = envec[eng_title].astype('float32').reshape(1, -1)
    faiss.normalize_L2(query_vec)

    # Retrieve the top 10 nearest neighbors:
    distances, indices = index_fr.search(query_vec, 10)
    retrieved_words = [french_words[i] for i in indices[0]]

    rank = 0
    for i, candidate in enumerate(retrieved_words):
        if candidate == fr_title:
            rank = i + 1
            break
    if rank:
        correct_retrievals += 1
        mrr_sum += 1.0 / rank

recall_at_10_fr = correct_retrievals / eligible_count if eligible_count else 0
mrr_at_10_fr = mrr_sum / eligible_count if eligible_count else 0

print("French recall@10:", recall_at_10_fr)
print("French MRR@10:", mrr_at_10_fr)


French recall@10: 0.9456681350954479
French MRR@10: 0.8830716966179522


## Speeding up Vector Search (required for CS6200, 20 points extra for IS4200)

The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.

A relatively easy way to get started with fast vector similarity search is to install Meta's `faiss` (Facebook AI Similarity Search) package and read [the tutorial](https://github.com/facebookresearch/faiss/wiki/Getting-started).

In [37]:
# Outside of colab, you may need a different package manager.
# !apt install libomp-dev
#!pip install --upgrade faiss-cpu
# Use this line instead if you have a GPU.
#!python -m pip install --upgrade faiss-gpu
import time
import numpy as np
import faiss

def evaluate_retrieval(test_set, envec, target_vec, target_words, method='linear', index=None):
    correct = 0
    total = 0
    mrr = 0.0
    start = time.time()

    for entry in test_set:
        eng_title = link_entry[0]
        target_title = entry[2]
        if eng_title not in envec or target_title not in target_vec:
            continue
        total += 1

        # For brute-force search:
        if method == 'linear':
            sims = [(word, cosim(envec[eng_title], vec)) for word, vec in target_vec.items()]
            sims.sort(key=lambda x: x[1], reverse=True)
            neighbors = [w for w, s in sims[:10]]

        # For FAISS-based search:
        elif method == 'faiss':
            query = envec[eng_title].astype('float32').reshape(1, -1)
            faiss.normalize_L2(query)
            _, idx = index.search(query, 10)
            neighbors = [target_words[i] for i in idx[0]]
        else:
            raise ValueError("Unknown method: " + method)

        # Determine rank of the correct target headword
        rank = 0
        for i, word in enumerate(neighbors):
            if word == target_title:
                rank = i + 1
                break
        if rank:
            correct += 1
            mrr += 1.0 / rank

    exec_time = time.time() - start
    recall_at_10 = correct / total if total else 0
    mrr_at_10 = mrr / total if total else 0
    return recall_at_10, mrr_at_10, exec_time

# French index
french_words = list(frvec.keys())
french_matrix = np.stack([frvec[w] for w in french_words]).astype('float32')
faiss.normalize_L2(french_matrix)
index_fr = faiss.IndexFlatIP(french_matrix.shape[1])
index_fr.add(french_matrix)

# Spanish index
spanish_words = list(esvec.keys())
spanish_matrix = np.stack([esvec[w] for w in spanish_words]).astype('float32')
faiss.normalize_L2(spanish_matrix)
index_es = faiss.IndexFlatIP(spanish_matrix.shape[1])
index_es.add(spanish_matrix)

fr_test = [entry for entry in links if entry[1] == "fr"][:1000]
es_test = [entry for entry in links if entry[1] == "es"][:1000]

# French: Brute-force
recall_lin_fr, mrr_lin_fr, time_lin_fr = evaluate_retrieval(fr_test, envec, frvec, french_words, method='linear')
# French: FAISS
recall_faiss_fr, mrr_faiss_fr, time_faiss_fr = evaluate_retrieval(fr_test, envec, frvec, french_words, method='faiss', index=index_fr)

print("French Brute-Force: Recall@10 = {:.4f}, MRR@10 = {:.4f}, Time = {:.4f} sec".format(recall_lin_fr, mrr_lin_fr, time_lin_fr))
print("French FAISS:       Recall@10 = {:.4f}, MRR@10 = {:.4f}, Time = {:.4f} sec".format(recall_faiss_fr, mrr_faiss_fr, time_faiss_fr))

# Spanish: Brute-force
recall_lin_es, mrr_lin_es, time_lin_es = evaluate_retrieval(es_test, envec, esvec, spanish_words, method='linear')
# Spanish: FAISS
recall_faiss_es, mrr_faiss_es, time_faiss_es = evaluate_retrieval(es_test, envec, esvec, spanish_words, method='faiss', index=index_es)

print("Spanish Brute-Force: Recall@10 = {:.4f}, MRR@10 = {:.4f}, Time = {:.4f} sec".format(recall_lin_es, mrr_lin_es, time_lin_es))
print("Spanish FAISS:       Recall@10 = {:.4f}, MRR@10 = {:.4f}, Time = {:.4f} sec".format(recall_faiss_es, mrr_faiss_es, time_faiss_es))

French Brute-Force: Recall@10 = 0.0029, MRR@10 = 0.0020, Time = 197.1706 sec
French FAISS:       Recall@10 = 0.0029, MRR@10 = 0.0020, Time = 2.3960 sec
Spanish Brute-Force: Recall@10 = 0.0015, MRR@10 = 0.0008, Time = 187.8518 sec
Spanish FAISS:       Recall@10 = 0.0015, MRR@10 = 0.0008, Time = 2.1614 sec


**TODO**: Create two vector indexes, for the FastText embeddings of French and for the other language you evaluated above. Use `faiss` to find the 10 nearest neighbors for the top 1000 Wikipedia headwords you evaluated for English-French and the English-X as above.

First, measure the _effectiveness_ of this approximate vector search approach. How does the R@10 and MRR@10 using `faiss` compare to the brute-force search you did above?

Second, measure the _efficiency_ of this approach. How long in seconds does finding nearest neighbors for 1000 headwords by brute force compare to using `faiss`? (For this exercise, don't worry about amortizing indexing costs.)